# Starbucks Capstone Challenge

## Business Understanding

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

## Data Understanding

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

## Gather

In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
# Change default settings to allow seeing all of the data in a table.
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200
pd.set_option('display.max_colwidth', 200)

## Assess

In [3]:
# This function identifies the percentage of missing values in a dataframe

def get_missing_pct(file_name):
    
    '''
    Inputs:  Enter the dataframe name that you want to assess for missing values.
    
    Outputs:  The function will output the variable names and percentage missing.
    
    '''
    
    percent_missing_filename = file_name.isnull().sum()*100 /len(file_name)
    missing_values_filename = pd.DataFrame({'column_name': file_name.columns,
                                 'percent_missing': percent_missing_filename})
    missing_values = missing_values_filename.sort_values('percent_missing', ascending=False)
    
    return missing_values

In [4]:
# View the contents of the portfolio dataframe (note:  it is only 10 rows in length)
portfolio.head(10)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [5]:
# A more detailed look at the portfolio data
portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


In [6]:
# check for missing values
get_missing_pct(portfolio)

,column_name,percent_missing
reward,reward,0.0
channels,channels,0.0
difficulty,difficulty,0.0
duration,duration,0.0
offer_type,offer_type,0.0
id,id,0.0


In [7]:
# Look at the structure of the portfolio dataframe
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


There are to be 10 rows in the portfolio dataframe.  They are displayed above.  The data structure appears to be correct. Difficulty (amount spent to achieve promotion) ranges from 0 to 20 dollars.  Offers appear to last between 3 and 7 days duration.  Portfolio is not missing any data.

#### Issues with the portfolio dataframe

* channels have multiple items.  We should break this into separate columns.<cr>
* Separate the offer types into individual columns
* Consider changing the portfolio ID so it is between 1 and 10 to make for better visibility
* Rename id to offer_id as it can be confused with the id in the profile dataframe


In [8]:
# View the contents of the portfolio dataframe (note:  it is only 10 rows in length)
profile.head(5)

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [9]:
# A more detailed look at the profile dataframe
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [10]:
# check for missing values
get_missing_pct(profile)

,column_name,percent_missing
gender,gender,12.794118
income,income,12.794118
age,age,0.000000
id,id,0.000000
became_member_on,became_member_on,0.000000


In [11]:
# Look at the structure of the profile dataframe
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


There are 17,000 rows in the profile dataframe.  The maximum age is showing as 118.  It is likely an outlier.  Looking at the structure of the data, there appear to be some missing values for gender and income.

#### Issues with the profile dataframe

* Some participants appear to have an excessive age of 118.  Investigate further.<cr>
* became_member_on should be converted to datetime<cr>
* Some gender are showing as None <cr>
* We should consider simplifying id to be a number starting at 1<cr>
* Income has some NaNs.  Will need to consider how to treat these NaNs.
* became_member_on should be converted to datetime

In [12]:
# View the contents of the transcript dataframe (note:  it is only 10 rows in length)
transcript.head(5)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [13]:
# A more detailed look at the transcript dataframe
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [14]:
# Look at the possible values for the event varible
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [15]:
# Look at possible values for the value field
transcript.value.sample(20)

203268                  {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
203212                  {'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'}
190699                                                 {'amount': 13.63}
229766                                                   {'amount': 7.7}
246128                  {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
77136                   {'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
111334                  {'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
265012                  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
119721                  {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
62252                   {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
47997                                                   {'amount': 2.87}
36505     {'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0', 'reward': 10}
268000                                                 {'amount': 26.71}
165383                  {'offer id': '2298d6c36e964

The value field contains a dictionary.  The key identifies if offer id is present it also contains an amount.  The value is either an offer_id, mapping to portfolio or an actual amount value.  These need to be moved into separate columns.  Also, notice that offer id sometimes appears as offer_id.

In [16]:
# Look at the structure of the transcript dataframe
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [17]:
# check for missing values
get_missing_pct(transcript)

,column_name,percent_missing
person,person,0.0
event,event,0.0
value,value,0.0
time,time,0.0


There are 306,534 rows in the transcript dataframe.  There do not appear to be any missing values.  Time ranges between 0 and 714 minutes.

#### Issues with the transcript dataframe

* Need to clean the value column of the "{'offer id': " and "}"<cr>
* Some rows in the value column contain a combination of the offer id and reward amount.  These will need to be separated and cleaned<cr>
* The value field also contains some amounts (spend).  These will also need to be cleaned and placed into their own separate column.


## Data Preparation

### Quality Issues

In [18]:
# make a copy of the data to make the changes
transcript_copy = transcript.copy(deep=False)
portfolio_copy = portfolio.copy(deep=False)
profile_copy = profile.copy(deep=False)

## Clean

In [19]:
# Identify the potential keys in the value column
value_key = []
for index, row in transcript.iterrows():
    for i in row['value']:
        if i in value_key:
            continue
        else:
            value_key.append(i)

print(value_key)

['offer id', 'amount', 'offer_id', 'reward']


In [20]:
# Expand the channel to have multiple columns for each channel
channels = portfolio_copy['channels'].str.join(sep='*').str.get_dummies(sep='*')
offer_type = pd.get_dummies(portfolio_copy['offer_type'])
df = pd.concat([portfolio_copy, channels, offer_type], axis=1, sort=False)
  
# Remove the channels and offer_type variables
df = df.drop(['channels', 'offer_type'], axis=1)
    
# Rename the columns
columns = ['id', 'difficulty', 'duration', 'reward', 'email', 'mobile', 'social',
           'web', 'bogo', 'discount', 'informational']
portfolio_clean = df[columns]


### Test

In [21]:
# View the portfolio_clean dataframe
portfolio_clean

,id,difficulty,duration,reward,email,mobile,social,web,bogo,discount,informational
0,ae264e3637204a6fb9bb56bc8210ddfd,10,7,10,1,1,1,0,1,0,0
1,4d5c57ea9a6940dd891ad53e9dbe8da0,10,5,10,1,1,1,1,1,0,0
2,3f207df678b143eea3cee63160fa8bed,0,4,0,1,1,0,1,0,0,1
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,7,5,1,1,0,1,1,0,0
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,10,5,1,0,0,1,0,1,0
5,2298d6c36e964ae4a3e7e9706d1fb8c2,7,7,3,1,1,1,1,0,1,0
6,fafdcd668e3743c1bb461111dcafc2a4,10,10,2,1,1,1,1,0,1,0
7,5a8bc65990b245e5a138643cd4eb9837,0,3,0,1,1,1,0,0,0,1
8,f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,1,1,1,1,1,0,0
9,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,0,1,0,1,0


In [22]:
# extract the dictionary and values into separate columns for amount, offer_id, and reward
transcript_copy = pd.concat([transcript_copy, transcript_copy['value'].apply(pd.Series)], axis=1)
transcript_copy.sample(10)

,person,event,value,time,offer id,amount,offer_id,reward
197720,ea9a6c23a601434c92072dfbc6483d57,transaction,{'amount': 20.3},486,NaN,20.30,NaN,NaN
34252,9431b531c73f402a810754e48c8eac0e,transaction,{'amount': 6.17},66,NaN,6.17,NaN,NaN
291116,0b83ad12131140a9bd1b2e5e1ae457e4,transaction,{'amount': 18.31},648,NaN,18.31,NaN,NaN
61221,12a442b1eef2476faee1829aa6e2e1b5,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},168,3f207df678b143eea3cee63160fa8bed,NaN,NaN,NaN
287490,0b680efe1a0a40788ebb6fb2c587b4a7,transaction,{'amount': 12.4},636,NaN,12.40,NaN,NaN
122755,829a8d14daf34f9299b21440c085ff49,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},336,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
77645,788db6a5814247e38bac0a3c0c56ec96,transaction,{'amount': 6.45},192,NaN,6.45,NaN,NaN
231802,64c7b16c67474a048f835ee40800ff2f,offer viewed,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},534,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
166468,6d78e73a854c433ca63effc96cedb6c0,offer completed,"{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}",408,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
247918,00c91f31f5f74e769fa7a359b63e1a9f,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},576,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN,NaN


In [23]:
#clean up dataset to unify multiple offer_id columns into one column
transcript_copy['offer_id']=np.where(transcript_copy['offer_id'].isnull(),
                                    transcript_copy['offer id'],transcript_copy['offer_id'])



In [24]:
# sort the transcript file by person and offer_id and save in transcript_sorted
transcript_sorted =transcript_copy.sort_values(['person', 'offer_id'])
transcript_sorted.head(20)

,person,event,value,time,offer id,amount,offer_id,reward
247879,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,2906b810c7d4411798c6938adc9daaa5,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
258884,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}",576,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
168413,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}",414,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,fafdcd668e3743c1bb461111dcafc2a4,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN


In [25]:
# forward fill offer_id to replace NaN.
transcript_sorted['offer_id'] = transcript_sorted['offer_id'].ffill()
transcript_sorted.head(20)

,person,event,value,time,offer id,amount,offer_id,reward
247879,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,2906b810c7d4411798c6938adc9daaa5,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
258884,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}",576,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
168413,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}",414,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,fafdcd668e3743c1bb461111dcafc2a4,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN


In [26]:
# resort transcript_copy by person, offer_id and time to arrange in the proper order
transcript_sorted =transcript_copy.sort_values(['person', 'offer_id', 'time'])
transcript_sorted.head(20)

,person,event,value,time,offer id,amount,offer_id,reward
247879,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,2906b810c7d4411798c6938adc9daaa5,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
258884,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}",576,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
168413,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}",414,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,fafdcd668e3743c1bb461111dcafc2a4,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN


In [27]:
# drop offer_id and value as they are no longer needed

transcript_copy.drop(['offer id', 'value'],axis=1,inplace=True)
transcript_copy.head()

,person,event,time,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


### Issue

Change portfolio id to numbers 1 through 10 and rename portfolio.id to portfolio.offer

In [28]:
def create_map_dicts(series):
    
    '''
    Create a mapping of values in a series to numbers.
    
    Args:
        series (Pandas series) - data in a series
        
    Returns:
        Mapping of the series to to integers as a dictionary
    '''
    
    iss, isn = dict(), 1
    for s in series.unique():
        iss[s] = int(isn)
        isn += 1
    return iss


def map_iss_to_num(series, map_dicts):
    
    '''
    Map the values in the series to new values from 1 to N
    
    Args:
        (1) series (Pandas series) - dataframe column to map
        (2) map_dicts (dicts) - utilizes the ouptut from the create_map_dict saved as a file (contains the mapping)
        
    Returns:
        Dataframe column with values mapped to integers (Pandas series).
    '''
    
    return series.map(map_dicts)

In [29]:
# use the create map dicts and map_iss_to_num to simplify portfolio
offer_mapping = create_map_dicts(portfolio_clean.id)
portfolio_clean['offer_number'] = map_iss_to_num(portfolio_clean.id, offer_mapping)

C:\Users\dsbb1\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
# drop id as it is no longer needed
portfolio_clean.drop(['id'], axis=1, inplace=True)

### Test

In [31]:
# verify that portfolio_clean has simple numbering for offer_number
portfolio_clean

,difficulty,duration,reward,email,mobile,social,web,bogo,discount,informational,offer_number
0,10,7,10,1,1,1,0,1,0,0,1
1,10,5,10,1,1,1,1,1,0,0,2
2,0,4,0,1,1,0,1,0,0,1,3
3,5,7,5,1,1,0,1,1,0,0,4
4,20,10,5,1,0,0,1,0,1,0,5
5,7,7,3,1,1,1,1,0,1,0,6
6,10,10,2,1,1,1,1,0,1,0,7
7,0,3,0,1,1,1,0,0,0,1,8
8,5,5,5,1,1,1,1,1,0,0,9
9,10,7,2,1,1,0,1,0,1,0,10


### Issue

Change the id in profile to a simple number from 1 to n and reanme it user_id

In [32]:
# use the create map dicts and map_iss_to_num to simplify user_id
user_mapping = create_map_dicts(profile_copy.id)
profile_copy['user_id'] = map_iss_to_num(profile_copy.id, user_mapping)

In [33]:
# drop id as it is no longer needed
profile_copy.drop(['id'], axis=1, inplace=True)

### Test

In [34]:
# verify user_id is simple numbering
profile_copy.head()

,gender,age,became_member_on,income,user_id
0,None,118,20170212,NaN,1
1,F,55,20170715,112000.0,2
2,None,118,20180712,NaN,3
3,F,75,20170509,100000.0,4
4,None,118,20170804,NaN,5


### Issue

Replace person with a simple user_id number from 1 to n.

In [35]:
# use the mapping alreadh created for offer_id and user_id to simplify thes in the transcript_sorted file
transcript_sorted['user_id'] = map_iss_to_num(transcript_sorted.person, user_mapping)
transcript_sorted['offer_number'] = map_iss_to_num(transcript_sorted.offer_id, offer_mapping)
transcript_sorted['offer_number'] = transcript_sorted.offer_number.fillna(0)
transcript_sorted['offer_number'] = transcript_sorted['offer_number'].astype('int64')

### Test

In [36]:
# verify the simplified user_id, offer_umber
transcript_sorted.head(10)

,person,event,value,time,offer id,amount,offer_id,reward,user_id,offer_number
247879,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,2906b810c7d4411798c6938adc9daaa5,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,3794,10
258884,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}",576,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0,3794,10
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN,3794,3
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN,3794,3
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN,3794,8
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN,3794,8
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN,3794,9
168413,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}",414,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0,3794,9
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN,3794,9
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,fafdcd668e3743c1bb461111dcafc2a4,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,3794,7


In [37]:
# drop person, value, offer_id and offer_id as they are no longer necessary
transcript_sorted.drop(['person', 'value', 'offer_id', 'offer id'], axis=1, inplace=True)

In [38]:
# verify the simplification worked and the columns were dropped
transcript_sorted.head(20)

,event,time,amount,reward,user_id,offer_number
247879,offer received,576,NaN,NaN,3794,10
258884,offer completed,576,NaN,2.0,3794,10
113605,offer received,336,NaN,NaN,3794,3
139992,offer viewed,372,NaN,NaN,3794,3
55972,offer received,168,NaN,NaN,3794,8
77705,offer viewed,192,NaN,NaN,3794,8
153401,offer received,408,NaN,NaN,3794,9
168413,offer completed,414,NaN,5.0,3794,9
187554,offer viewed,456,NaN,NaN,3794,9
204340,offer received,504,NaN,NaN,3794,7


In [39]:
# set NaN values to zero
transcript_sorted['amount']=np.where(transcript_sorted['amount'].isnull(),
                                    0,transcript_sorted['amount'])
transcript_sorted['reward']=np.where(transcript_sorted['reward'].isnull(),
                                    0,transcript_sorted['reward'])



In [40]:
# verify that NaN values were addressed
transcript_sorted.isnull().sum()

event           0
time            0
amount          0
reward          0
user_id         0
offer_number    0
dtype: int64

In [41]:
# create function to create new columns and populate a 1 or zero based on the event
def create_steps(df, text, col_name):
    text_to_search = text
    new_col_name = col_name
    df[new_col_name] = df.event.apply(lambda x: 1 if x == text_to_search else 0)

In [42]:
# execute the function for the events (offer received, offer viewed, transaction, and offer completed)
create_steps(transcript_sorted, 'offer received', 'offer_received')
create_steps(transcript_sorted, 'offer viewed', 'offer_viewed')
create_steps(transcript_sorted, 'transaction', 'transaction')
create_steps(transcript_sorted, 'offer completed', 'offer_completed')

In [43]:
transcript_sorted.head(20)

,event,time,amount,reward,user_id,offer_number,offer_received,offer_viewed,transaction,offer_completed
247879,offer received,576,0.00,0.0,3794,10,1,0,0,0
258884,offer completed,576,0.00,2.0,3794,10,0,0,0,1
113605,offer received,336,0.00,0.0,3794,3,1,0,0,0
139992,offer viewed,372,0.00,0.0,3794,3,0,1,0,0
55972,offer received,168,0.00,0.0,3794,8,1,0,0,0
77705,offer viewed,192,0.00,0.0,3794,8,0,1,0,0
153401,offer received,408,0.00,0.0,3794,9,1,0,0,0
168413,offer completed,414,0.00,5.0,3794,9,0,0,0,1
187554,offer viewed,456,0.00,0.0,3794,9,0,1,0,0
204340,offer received,504,0.00,0.0,3794,7,1,0,0,0


In [44]:
# summarize the rows of data for each user_id, offer_number so 1 row exists for each user_id and offer_number
transcript_user_summary = transcript_sorted.groupby(['user_id', 'offer_number']).agg({'offer_received': ['max'],
                                                                            'offer_viewed': ['max'],
                                                                             'transaction': ['sum'],
                                                                             'reward': ['sum'],'amount': ['sum'],
                                                                                      
                                                                             'offer_completed': ['max']}).reset_index()

### Test

In [45]:
# verify the summary
transcript_user_summary.sample(20)

,user_id,offer_number,offer_received,offer_viewed,transaction,reward,amount,offer_completed
,,,max,max,sum,sum,sum,max
58055,12334,0,0,0,11,0.0,24.25,0
41599,8841,0,0,0,8,0.0,179.51,0
54750,11629,7,1,1,0,2.0,0.00,1
76105,16203,10,1,0,0,0.0,0.00,0
55172,11718,0,0,0,4,0.0,80.77,0
73463,15638,9,1,1,0,5.0,0.00,1
75950,16171,0,0,0,7,0.0,117.59,0
60368,12834,8,1,1,0,0.0,0.00,0
820,175,4,1,1,0,5.0,0.00,1


In [46]:
# look at the structure of the data
transcript_user_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79866 entries, 0 to 79865
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   (user_id, )             79866 non-null  int64  
 1   (offer_number, )        79866 non-null  int64  
 2   (offer_received, max)   79866 non-null  int64  
 3   (offer_viewed, max)     79866 non-null  int64  
 4   (transaction, sum)      79866 non-null  int64  
 5   (reward, sum)           79866 non-null  float64
 6   (amount, sum)           79866 non-null  float64
 7   (offer_completed, max)  79866 non-null  int64  
dtypes: float64(2), int64(6)
memory usage: 4.9 MB


In [48]:
transcript_user_summary.head()

,user_id,offer_number,offer_received,offer_viewed,transaction,reward,amount,offer_completed
,,,max,max,sum,sum,sum,max
0,1,0,0,0,9,0.0,20.4,0
1,1,5,1,1,0,0.0,0.0,0
2,1,6,1,1,0,3.0,0.0,1
3,1,7,1,1,0,2.0,0.0,1
4,1,10,1,1,0,0.0,0.0,0


In [49]:
# look at summary statistics for transcript_user_summary.  Allows verification that offer_received, offer_viewed,
# and offer_completed have only 1 and 0 as their possible values
transcript_user_summary.describe()

,user_id,offer_number,offer_received,offer_viewed,transaction,reward,amount,offer_completed
,,,max,max,sum,sum,sum,max
count,79866.000000,79866.000000,79866.000000,79866.000000,79866.000000,79866.000000,79866.000000,79866.000000
mean,8494.927378,4.349273,0.792427,0.615218,1.739827,2.061904,22.230386,0.363058
std,4902.583777,3.388702,0.405572,0.486547,4.094564,3.656122,72.128651,0.480884
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4248.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8488.500000,4.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,12734.000000,7.000000,1.000000,1.000000,0.000000,3.000000,0.000000,1.000000
max,17000.000000,10.000000,1.000000,1.000000,36.000000,40.000000,1608.690000,1.000000


Group 1 - 
* Offer is BOGO or Discount and user completes all components (offer received, offer viewed, transaction, offer completed)
* Offer is informational and user completes the following 3 components (offer received, offer viewed, transaction)

Group 2 - 
* Offer contains the following components only (offer received, offer viewed)

Group 4 - 
* Offer contains the following components only (offer received)

Group - 
* This is the most complicated.  The easiest way to solve this was to set the False value for all other groups to 3.  If they were not overwritten, group must equal 3.
         

In [50]:
# this defines group 1 for BOGO and Discounts
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([1,2,4,5,6,7,9,10])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 1) & 
                                    (transcript_user_summary['transaction'] == 1) &                                     
                                    (transcript_user_summary['offer_completed'] == 1),1,3)

MemoryError: Unable to allocate 47.5 GiB for an array with shape (79866, 79866) and data type float64

In [ ]:
# this defines group 1 for informational promos
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([3,8])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 1) & 
                                    (transcript_user_summary['transaction'] == 1) &                                     
                                    (transcript_user_summary['offer_completed'] == 0),1,3)

In [ ]:
# this defines group 2 for all promos
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([1,2,3,4,5,6,7,8,9,10])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 1) & 
                                    (transcript_user_summary['transaction'] == 0) &                                     
                                    (transcript_user_summary['offer_completed'] == 0),2,3)

In [ ]:
# this defines group 4 for all promos
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([1,2,3,4,5,6,7,8,9,10])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 0) & 
                                    (transcript_user_summary['transaction'] == 0) &                                     
                                    (transcript_user_summary['offer_completed'] == 0),4,3)

## Visualize

## Model

### Data Preparation for modeling

### Predicting Model

## Evaluation

In [ ]:
# place the values from teh dictionaries into the columns


for index, row in transcript_copy.iterrows():
    
    for i in row['value']:
        if i == 'reward':
            transcript_copy.at[index, 'reward'] = row['value'][i]
        if i == 'amount':
            transcript_copy.at[index, 'amount'] = row['value'][i]
        if i == 'offer id' or i == 'offer_id': 
            transcript_copy.at[index, 'offer_id'] = row['value'][i]



In [ ]:
# create additional columns to house the reward, amount and offer
transcript_sorted['reward'] = 0
transcript_sorted['amount'] = float(0)
